# TCN Model on Lab p4679
Author: Camilla Billari <br> 
Date: 07/02/24

Fitting a TCN to the lab p4679 dataset.

In [1]:
# Import relevant libraries
import matplotlib.pyplot as plt

# Import local modules - note: dependent on above path being set.
from utils.dataset import SlowEarthquakeDataset
# from utils.nn_io import load_model, save_model
from utils.general_functions import set_seed, set_torch_device
from utils.data_preprocessing import moving_average_causal_filter, compare_feature_statistics, create_dataset, split_train_test_forecast_windows, normalise_dataset
from utils.plotting import plot_original_vs_processed_data, plot_example_sample, plot_metric_results, plot_all_data_results
from utils.nn_train import train_model
from scripts.train_cascadia_with_optuna import run_optuna_optimization
from scripts.models.tcn_oneshot_multistep import MultiStepTCN

In [2]:
# Set random seed
SEED = 17 # random seed for the dataset and model
set_seed(SEED)

# Set torch device
device = set_torch_device()

No GPU available.


## Load Data

In [3]:
run_optuna_optimization()


[I 2024-02-20 17:29:04,162] A new study created in memory with name: no-name-1c3c6e2d-ca36-4b56-bc84-497d7fc0180a


Starting Optuna optimization with 5 trials...
No GPU available.
No GPU available.
No GPU available.
No GPU available.
Training model on cpu
Training model on cpu
Training model on cpu
Training model on cpu


Epoch [75/75], RMSE Train: 0.4568, RMSE Test: 0.4893: 100%|██████████| 75/75 [07:33<00:00,  6.04s/it]
[I 2024-02-20 17:36:41,679] Trial 1 finished with value: 0.4892915189266205 and parameters: {'n_layers': 4, 'hidden_size': 16, 'kernel_size': 2, 'dropout': 0.43211882690853165}. Best is trial 1 with value: 0.4892915189266205.
Epoch [68/75], RMSE Train: 0.1921, RMSE Test: 0.2591:  91%|█████████ | 68/75 [07:42<00:47,  6.73s/it]

No GPU available.
Training model on cpu


Epoch [75/75], RMSE Train: 0.2079, RMSE Test: 0.2464: 100%|██████████| 75/75 [08:28<00:00,  6.78s/it]
[I 2024-02-20 17:37:37,117] Trial 3 finished with value: 0.24641640484333038 and parameters: {'n_layers': 2, 'hidden_size': 16, 'kernel_size': 7, 'dropout': 0.24038181041138046}. Best is trial 3 with value: 0.24641640484333038.
Epoch [75/75], RMSE Train: 0.2355, RMSE Test: 0.2960: 100%|██████████| 75/75 [08:37<00:00,  6.90s/it]
[I 2024-02-20 17:37:46,082] Trial 2 finished with value: 0.29597580432891846 and parameters: {'n_layers': 2, 'hidden_size': 16, 'kernel_size': 8, 'dropout': 0.41733635223404986}. Best is trial 3 with value: 0.24641640484333038.
Epoch [75/75], RMSE Train: 0.2035, RMSE Test: 0.2655: 100%|██████████| 75/75 [19:46<00:00, 15.82s/it]
[I 2024-02-20 17:48:55,056] Trial 0 finished with value: 0.2654551863670349 and parameters: {'n_layers': 2, 'hidden_size': 128, 'kernel_size': 3, 'dropout': 0.48426566935899806}. Best is trial 3 with value: 0.24641640484333038.
Epoch [75/

Best trial:
  Value:  0.24641640484333038
  Params: 
    n_layers: 2
    hidden_size: 16
    kernel_size: 7
    dropout: 0.24038181041138046


dict_items([('n_layers', 2), ('hidden_size', 16), ('kernel_size', 7), ('dropout', 0.24038181041138046)])

In [ ]:
EXP = "p4679" # select Marone p4679 experiment

# Load dataset and convert to dataframe
dataset = SlowEarthquakeDataset([EXP])
df = SlowEarthquakeDataset.convert_to_df(dataset, EXP)

df.head()

In [ ]:
# Select segment average and scale down by 10^8
shear_stress_df = df["obs_shear_stress"]

shear_stress_df.shape

## Data Pre-processing

### Denoising

In [ ]:
# Data smoothing
SMOOTHING_WINDOW = 1 # moving average window size
DOWNSAMPLING_FACTOR = 100

# Smooth the data
df_smoothed = moving_average_causal_filter(shear_stress_df, SMOOTHING_WINDOW, DOWNSAMPLING_FACTOR)

df_smoothed.shape

In [ ]:
# Check if smoothed results are statistically significantly different
results = compare_feature_statistics(shear_stress_df, df_smoothed)
print(results)

In [ ]:
plot_original_vs_processed_data(shear_stress_df, df_smoothed, "line")

In [ ]:
plot_original_vs_processed_data(shear_stress_df, df_smoothed, "scatter")

## Creating the datastet

In [ ]:
# Dataset parameters
LOOKBACK, FORECAST = 90, 30 # lookback and forecast values
N_FORECAST_WINDOWS = 60 # n forecasted windows in test set

# Pre-process the data into windows
X, y = create_dataset(df_smoothed, LOOKBACK, FORECAST)

print(f"X and y shapes: {X.shape}, {y.shape}")

In [ ]:
# Split into train and test sets and normalise it
X_train, y_train, X_test, y_test = split_train_test_forecast_windows(X, y, FORECAST, N_FORECAST_WINDOWS)

# Display shapes of the created datasets
print(f"Training data shapes: {X_train.shape}, {y_train.shape}")
print(f"Testing data shapes: {X_test.shape}, {y_test.shape}")

### Normalising with scaler based on the training set

In [ ]:
# Normalise the dataset and save all sets into data_dict; keep scalers for later.
data_dict, scaler_X, scaler_y = normalise_dataset(X_train, y_train, X_test, y_test)

print(data_dict.keys())

In [ ]:
# Plot 3rd X_test window
plt.plot(data_dict["X_test_sc"][2, :])

In [ ]:
plot_example_sample(data_dict["X_test_sc"], data_dict["y_test_sc"], 10, LOOKBACK, FORECAST)

## Modelling

In [ ]:
# For TCN config
N_VARIATES = 1
HIDDEN_SIZE = 50
KERNEL_SIZE = 3
OUTPUT_SIZE = FORECAST
DROPOUT = 0

model = MultiStepTCN(N_VARIATES, LOOKBACK, OUTPUT_SIZE, [HIDDEN_SIZE], KERNEL_SIZE,  DROPOUT).to(device)

### Training

In [ ]:
# For TCN training
N_EPOCHS = 75
results_dict = train_model(model, N_EPOCHS, data_dict, scaler_y, device)

## Results

In [ ]:
# Plot RMSE
plot_metric_results(N_EPOCHS, results_dict["train_rmse_list"], results_dict["test_rmse_list"], "RMSE")

In [ ]:
# Plot R2
plot_metric_results(N_EPOCHS, results_dict["train_r2_list"], results_dict["test_r2_list"], "R$^2$")

In [ ]:
# Plot predictions against true values
TEST_START_IDX = len(df_smoothed) - len(y_test) - LOOKBACK
TITLE = "Original Time Series and Model Predictions of Segment 1 sum"
X_LABEL = "Time (days)"
Y_LABEL = "Displacement potency (?)"
ZOOM_MIN = 2600
ZOOM_MAX = 2800
ZOOM_WINDOW = [ZOOM_MIN, ZOOM_MAX]

In [ ]:
plot_all_data_results(TEST_START_IDX, data_dict, results_dict, LOOKBACK, FORECAST, TITLE, X_LABEL, Y_LABEL, [])


In [ ]:
plot_all_data_results(TEST_START_IDX, data_dict, results_dict, LOOKBACK, FORECAST, TITLE, X_LABEL, Y_LABEL, ZOOM_WINDOW)
# LOOK AT BOKE